
# Price Elasticity & Price Optimization — Log-Log Model

This notebook extends the original analysis with:
- **Exploratory Data Analysis (EDA)** (distribution, spread, correlations, outliers)
- **Log normalization** for `Price` and `Quantity`
- **Demand modeling** in both **levels** and **log–log** (constant elasticity)
- **PED estimation** and **profit-optimizing price** computation
- Clear, single-purpose **visuals** (matplotlib only)

> Expected CSV: **`price_data.csv`** with columns **`Price`** and **`Quantity`**.


## 1) Setup

In [ ]:

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.api import jarque_bera

# Chart defaults (one chart per cell; no explicit colors)
plt.rcParams['figure.figsize'] = (7,4)
plt.rcParams['axes.grid'] = True

print("Libraries ready.")


## 2) Load dataset

In [ ]:

from pathlib import Path

CANDIDATES = [Path('price_data.csv'), Path('./data/price_data.csv')]
DATA_PATH = next((p for p in CANDIDATES if p.exists()), None)

if DATA_PATH is None:
    # Fallback: synthesize a dataset so the notebook remains demonstrable
    rng = np.random.default_rng(42)
    n = 1000
    # price around 50-300 with outliers
    price = np.clip(rng.normal(180, 70, size=n), 15, 1000).round(0)
    # quantity: inverse-ish relation + noise, intentionally wide spread
    quantity = (5000 * (price**-0.9) * np.exp(rng.normal(0, 0.7, size=n))).round(0)
    df = pd.DataFrame({'Price': price.astype(int), 'Quantity': quantity.astype(int)})
    DATA_PATH = Path('price_data_synthetic.csv')
    df.to_csv(DATA_PATH, index=False)
    print(f"⚠️ Could not find `price_data.csv`. Generated a synthetic demo dataset at: {DATA_PATH}")
else:
    df = pd.read_csv(DATA_PATH)
    print(f"Using: {DATA_PATH}")

assert {'Price','Quantity'}.issubset(df.columns), "CSV must have Price and Quantity columns"
df.head()


## 3) Exploratory Data Analysis

In [ ]:

summary = df.describe()
print(summary)

# Basic sanity checks
n_zeros = (df[['Price','Quantity']]==0).sum()
n_neg = (df[['Price','Quantity']]<0).sum()
print("\nZeros per column:\n", n_zeros)
print("\nNegatives per column:\n", n_neg)


In [ ]:

# Histogram: Price
plt.figure()
plt.hist(df['Price'].dropna(), bins=40)
plt.title("Histogram — Price")
plt.xlabel("Price")
plt.ylabel("Count")
plt.show()


In [ ]:

# Histogram: Quantity (note likely right-skew & heavy tail)
plt.figure()
plt.hist(df['Quantity'].dropna(), bins=40)
plt.title("Histogram — Quantity")
plt.xlabel("Quantity")
plt.ylabel("Count")
plt.show()


In [ ]:

# Scatter (levels)
plt.figure()
plt.scatter(df['Price'], df['Quantity'], s=10)
plt.title("Scatter — Quantity vs Price (levels)")
plt.xlabel("Price")
plt.ylabel("Quantity")
plt.show()


In [ ]:

corr = df[['Price','Quantity']].corr()
print("Correlation matrix:\n", corr)


## 4) Log normalization

In [ ]:

# Ensure strictly positive before log
mask_pos = (df['Price']>0) & (df['Quantity']>0)
df_log = df.loc[mask_pos, ['Price','Quantity']].copy()
df_log['lnPrice'] = np.log(df_log['Price'])
df_log['lnQty'] = np.log(df_log['Quantity'])

print(df_log[['Price','Quantity','lnPrice','lnQty']].head())
print("\nDescribe (logged):\n", df_log[['lnPrice','lnQty']].describe())


In [ ]:

# Histograms on the log scale
plt.figure()
plt.hist(df_log['lnPrice'], bins=40)
plt.title("Histogram — ln(Price)")
plt.xlabel("ln(Price)")
plt.ylabel("Count")
plt.show()

plt.figure()
plt.hist(df_log['lnQty'], bins=40)
plt.title("Histogram — ln(Quantity)")
plt.xlabel("ln(Quantity)")
plt.ylabel("Count")
plt.show()


## 5) Demand models

In [ ]:

# (A) Linear levels: Q = a + b*P
mod_lin = smf.ols('Quantity ~ Price', data=df).fit()
print(mod_lin.summary())
a, b = mod_lin.params['Intercept'], mod_lin.params['Price']

# PED at sample means (point elasticity for linear demand)
P_bar = df['Price'].mean()
Q_bar = df['Quantity'].mean()
ped_at_mean = b * (P_bar / Q_bar)
print(f"\nLinear model PED at means = {ped_at_mean:.4f}")


In [ ]:

# (B) Log–log: ln Q = α + β ln P  (β is elasticity)
mod_log = smf.ols('lnQty ~ lnPrice', data=df_log).fit()
print(mod_log.summary())
beta = mod_log.params['lnPrice']
print(f"\nLog–log elasticity (β) = {beta:.4f}")


In [ ]:

# Visual: levels with fitted line
plt.figure()
plt.scatter(df['Price'], df['Quantity'], s=10, alpha=0.7)
x = np.linspace(df['Price'].min(), df['Price'].max(), 100)
y_hat = a + b * x
plt.plot(x, y_hat)
plt.title("Demand (levels) with OLS fit")
plt.xlabel("Price")
plt.ylabel("Quantity")
plt.show()


In [ ]:

# Visual: log–log with fitted line
plt.figure()
plt.scatter(df_log['lnPrice'], df_log['lnQty'], s=10, alpha=0.7)
xx = np.linspace(df_log['lnPrice'].min(), df_log['lnPrice'].max(), 100)
yy_hat = mod_log.params['Intercept'] + beta * xx
plt.plot(xx, yy_hat)
plt.title("Demand (log–log) with OLS fit")
plt.xlabel("ln(Price)")
plt.ylabel("ln(Quantity)")
plt.show()


## 6) Residual diagnostics

In [ ]:

# Breusch–Pagan for heteroskedasticity (levels)
bp_stat, bp_p, _, _ = het_breuschpagan(mod_lin.resid, mod_lin.model.exog)
print(f"Breusch–Pagan (levels): stat={bp_stat:.3f}, p={bp_p:.4g}")

# Jarque–Bera for residual normality (levels)
jb_stat, jb_p, _, _ = jarque_bera(mod_lin.resid)
print(f"Jarque–Bera (levels): stat={jb_stat:.3f}, p={jb_p:.4g}")

# Breusch–Pagan for log–log
bp_stat2, bp_p2, _, _ = het_breuschpagan(mod_log.resid, mod_log.model.exog)
print(f"Breusch–Pagan (log–log): stat={bp_stat2:.3f}, p={bp_p2:.4g}")

# Jarque–Bera for log–log
jb_stat2, jb_p2, _, _ = jarque_bera(mod_log.resid)
print(f"Jarque–Bera (log–log): stat={jb_stat2:.3f}, p={jb_p2:.4g}")


In [ ]:

# Residual plots: one chart per cell

# Levels
plt.figure()
plt.scatter(mod_lin.fittedvalues, mod_lin.resid, s=10)
plt.axhline(0, linestyle="--")
plt.title("Residuals vs Fitted — levels")
plt.xlabel("Fitted")
plt.ylabel("Residuals")
plt.show()


In [ ]:

# Log–log
plt.figure()
plt.scatter(mod_log.fittedvalues, mod_log.resid, s=10)
plt.axhline(0, linestyle="--")
plt.title("Residuals vs Fitted — log–log")
plt.xlabel("Fitted")
plt.ylabel("Residuals")
plt.show()


## 7) Price optimization (profit) with log–log model

In [ ]:

# In a constant-elasticity model Q(P) = A * P^β (β < 0), profit π = (P - c) * Q(P).
# First-order condition -> optimal price P* = c * η / (η - 1), where η = -β (>0).
beta = float(mod_log.params['lnPrice'])
eta = -beta

# User-set unit cost (USD). Adjust this to your case:
cost_per_unit = 3.0

if eta <= 1:
    print("⚠️ Estimated elasticity implies η ≤ 1. Profit optimum is not interior; check model/data.")
    P_star = np.nan
else:
    P_star = cost_per_unit * eta / (eta - 1)

A = float(np.exp(mod_log.params['Intercept']))
def Q_of_P(p): return A * (p ** beta)

# Evaluate at a grid for visuals
gridP = np.linspace(max(0.5, cost_per_unit*0.8), max(df['Price'].max(), cost_per_unit*5), 200)
rev = gridP * Q_of_P(gridP)
prof = (gridP - cost_per_unit) * Q_of_P(gridP)

print(f"Elasticity η = {-beta:.4f}")
print(f"Cost per unit c = {cost_per_unit:.2f}")
print(f"Suggested optimal price P* ≈ {P_star:.2f}") if not np.isnan(P_star) else None


In [ ]:

# Visual: Revenue curve
plt.figure()
plt.plot(gridP, rev)
plt.title("Revenue curve from log–log demand")
plt.xlabel("Price")
plt.ylabel("Revenue")
plt.show()


In [ ]:

# Visual: Profit curve with indicated P*
plt.figure()
plt.plot(gridP, prof)
if 'P_star' in locals() and np.isfinite(P_star):
    plt.axvline(P_star, linestyle="--")
    plt.title(f"Profit curve (c={cost_per_unit:.2f}), P*≈{P_star:.2f}")
else:
    plt.title(f"Profit curve (c={cost_per_unit:.2f}) — no interior optimum")
plt.xlabel("Price")
plt.ylabel("Profit")
plt.show()
